# Imports

In [1]:
import os
os.chdir("D:\PulpitE\FPL_ML")

In [2]:
import pandas as pd

import torch
import torch.nn as nn

import torchvision
import torchvision.transforms

from torch.utils.data import Dataset, DataLoader
import torch.optim as optim

import matplotlib.pyplot as plt

import numpy as np
import random

# Reading data

In [3]:
df = pd.read_csv('data/fpl_fbref_elo_players.csv')

In [4]:
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

# New features

In [5]:
# average goals for and against team in last 5 matches
grouped = df.groupby('Name')

In [6]:
df['ScoreForLast5'] = grouped['Team Score'].rolling(5, min_periods=1).mean().reset_index(level=0, drop=True)

In [7]:
df['ScoreAgainstLast5'] = grouped['Opp Score'].rolling(5, min_periods=1).mean().reset_index(level=0, drop=True)

In [8]:
df['ScoreForLast5'] = df['ScoreForLast5'].shift(1)
df['ScoreAgainstLast5'] = df['ScoreAgainstLast5'].shift(1)

In [9]:
df[df["Name"] == "Mohamed-Salah"][["ScoreForLast5", "ScoreAgainstLast5", "Team Score", "Opp Score"]]

,ScoreForLast5,ScoreAgainstLast5,Team Score,Opp Score
5896,2.400000,1.000000,2,2
5897,2.000000,2.000000,1,1
5898,1.500000,1.500000,1,2
5899,1.333333,1.666667,9,0
5900,3.250000,1.250000,2,1
5901,3.000000,1.200000,0,0
5902,2.600000,0.800000,3,3
5903,3.000000,1.200000,2,3
5904,3.200000,1.400000,1,0
5905,1.600000,1.400000,1,0


# Saving df to file

In [10]:
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

In [11]:
df.to_csv('data/final_dataset.csv')

In [12]:
df

,Date,Day,Comp,Round,Venue,Squad,Opponent,Start,Pos,Min,...,Was Home,GW,FPL,neutral_name,Total Points,FPL position,Opp rating,Team rating,ScoreForLast5,ScoreAgainstLast5
0,2022-08-06,Sat,Premier League,Matchweek 1,Home,Leeds United,Wolves,Y,RW,83.0,...,1.0,1,2.0,Brenden-Aaronson,2.0,MID,1725.276489,1703.435669,NaN,NaN
1,2022-08-13,Sat,Premier League,Matchweek 2,Away,Leeds United,Southampton,Y,"RW,AM",90.0,...,0.0,2,2.0,Brenden-Aaronson,1.0,MID,1667.049927,1710.825562,2.000000,1.0
2,2022-08-21,Sun,Premier League,Matchweek 3,Home,Leeds United,Chelsea,Y,AM,82.0,...,1.0,3,8.0,Brenden-Aaronson,8.0,MID,1901.388306,1711.222290,2.000000,1.5
3,2022-08-27,Sat,Premier League,Matchweek 4,Away,Leeds United,Brighton,Y,"AM,RW",81.0,...,0.0,4,2.0,Brenden-Aaronson,1.0,MID,1783.540161,1731.332397,2.333333,1.0
4,2022-08-30,Tue,Premier League,Matchweek 5,Home,Leeds United,Everton,Y,"AM,RW",75.0,...,1.0,5,5.0,Brenden-Aaronson,5.0,MID,1688.922485,1725.978516,1.750000,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7452,2022-12-26,Mon,Premier League,Matchweek 17,Home,Arsenal,West Ham,Y*,RM,90.0,...,1.0,17,8.0,Martin-Odegaard,11.0,MID,1768.070923,1921.228271,2.000000,0.2
7453,2022-12-31,Sat,Premier League,Matchweek 18,Away,Arsenal,Brighton,Y*,RM,86.0,...,0.0,18,10.0,Martin-Odegaard,13.0,MID,1799.850708,1926.285278,2.400000,0.4
7454,2023-01-03,Tue,Premier League,Matchweek 19,Home,Arsenal,Newcastle Utd,Y*,RM,90.0,...,1.0,19,3.0,Martin-Odegaard,2.0,MID,1850.296631,1934.447754,3.000000,0.6
7455,2023-01-15,Sun,Premier League,Matchweek 20,Away,Arsenal,Tottenham,Y*,RM,89.0,...,0.0,20,8.0,Martin-Odegaard,9.0,MID,1868.577759,1931.196655,2.000000,0.6
